In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import os

In [3]:
cd /content/drive/MyDrive/Colab Notebooks/titanic

/content/drive/MyDrive/Colab Notebooks/titanic


In [4]:
train_data = pd.read_csv("input/train.csv")
test_data = pd.read_csv("input/test.csv")

In [5]:
y = train_data["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

In [6]:
model = RandomForestClassifier(n_estimators=100, max_depth=1, random_state=1)
model.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=1, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [7]:
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})